In [1]:
import os
import json
import re
import pandas as pd
from collections import OrderedDict

In [2]:
def get_end_salvo(versao, path = None):
    
    if path is None:
        path = '../Dados_gerados/geocode'
    if os.path.exists(path):
        pat = 'enderecos_georref_\d*.json'
        files = [os.path.join(path, file) for file in os.listdir(path)
                if re.match(pat, file)]

        for f in files:
            versao_file = os.path.split(f)[-1].split('_')[-1].split('.')[0]
            if str(versao) == versao_file:
                with open(f) as file:
                    return json.load(file)
    
    
    return None


def dados_banco_dados():
    
    path_dados = '../Dados_originais/Dados_Banco_de_Dados'
    files_site = [os.path.join(path_dados, file) for file in os.listdir(path_dados)]
    
    contrib_f = [f for f in files_site if f.split('\\')[-1].startswith('contrib')][0]
    prop_f = [f for f in files_site if f.split('\\')[-1].startswith('result')][0]
    
    return pd.read_csv(contrib_f), pd.read_csv(prop_f)

In [3]:
def get_component_by_type(comp, type_name):
    
    if type_name in comp['types']:
        return comp['long_name']
    else:
        return None

def parse_address_comps(address):
    
    de_para = OrderedDict(
        {
            'bairro' : 'sublocality_level_1',
            'cidade' : 'administrative_area_level_2',
            'estado' : 'administrative_area_level_1',
            'pais' : 'country',
            
        }
    )
    
    dados = {}
    for nom_meu, nom_google in de_para.items():
        #nao consegui pensar em uma otimizacao, infelizmente,
        # mas a ordem tende a seguir a do de-para
        for component in address['address_components']:
            parsed = get_component_by_type(component, nom_google)
            if parsed:
                dados[nom_meu] = parsed
                break
        else:
            dados[nom_meu] = None
    
    return dados

def parse_address(address):
    
    dados = {}
    dados.update(parse_address_comps(address))
    dados['endereco_completo'] = address['formatted_address']
    dados['tipo_endereco'] = '; '.join(address['types'])
    dados['latitude'] = address['geometry']['location']['lat']
    dados['longitude'] = address['geometry']['location']['lng']
    dados['tipo_localizacao'] = address['geometry']['location_type']
    
    return dados

def registro_vazio():
    
    return {
                'bairro' : None,
                'cidade' : None,
                'estado' : None,
                'pais' : None,
                'endereco_completo' : None,
                'tipo_endereco' : None,
                'latitude' : None,
                'longitude' : None,
                'tipo_localizacao' : None,
            }

def parse_all_geocode_data(enderecos):
    
    parsed_data = {}
    for end_original, results in enderecos.items():
        if len(results) == 1:
            parsed_data[end_original] = parse_address(results[0])
        elif len(results) > 1:
            print(end_original)
            #só tem um caso, achei ok pegar o primeiro resultado
            parsed_data[end_original] = parse_address(results[0])
        elif len(results) == 0:
            parsed_data[end_original] = registro_vazio()
        
    return parsed_data

def data_frame_geocod(parsed_data):
    
    return pd.DataFrame(parsed_data).T

In [4]:
enderecos = get_end_salvo(versao = 1)
contrib, prop = dados_banco_dados()

In [5]:
results = enderecos.values()
print(f'Total de enderecos : {len(results)}')
cidades = {}
for result in results:
    #pode ter mais de um endereço
    for end in result:
        for component in end['address_components']:
            if 'administrative_area_level_2' in component['types']:
                nom_cidade = component['long_name']
                if nom_cidade not in cidades:
                    cidades[nom_cidade] = 0
                cidades[nom_cidade] +=1
for key, value in cidades.items():
    print(f'{key} : {value}')

Total de enderecos : 254
São Paulo : 245
Guilford County : 1
São Vicente : 1
Bonif : 1
Osasco : 2
Guarulhos : 1
Jundiaí : 1


In [6]:
parsed = parse_all_geocode_data(enderecos)

Rua Henry Fuseli 70 Viela são pedro São Paulo SP


In [7]:
df_parsed = data_frame_geocod(parsed)

In [8]:
df_parsed.to_excel('../Dados_gerados/enderecos_geocod.xlsx')

In [10]:
df_parsed.head()

,bairro,cidade,estado,pais,endereco_completo,tipo_endereco,latitude,longitude,tipo_localizacao
Rua Cachoeira Camaleão 247 São Paulo SP,Conjunto Habitacional Inacio Monteiro,São Paulo,São Paulo,Brazil,"R. Cachoeira Camaleão, 247 - Conj. Hab. Inacio...",street_address,-23.572076,-46.402959,ROOFTOP
Rua Fani Lerner 30 São Paulo SP,None,São Paulo,São Paulo,Brazil,"R. Fani Lerner, 30 - Jaragua, São Paulo - SP, ...",premise,-23.434769,-46.740747,GEOMETRIC_CENTER
Rua Albertina Vieira da Silva Gordo 212 apto 51 bl 1 São Paulo SP,Vila Aurora (Zona Norte),São Paulo,São Paulo,Brazil,"R. Albertina Viêira da Silva Gordo, 212 - apto...",subpremise,-23.476999,-46.623821,ROOFTOP
Rua Araújo 124 São Paulo SP,República,São Paulo,São Paulo,Brazil,"R. Araújo, 124 - República, São Paulo - SP, 01...",street_address,-23.54491,-46.644696,ROOFTOP
Rua Doutor Mário Vicente 1725 São Paulo SP,Vila Dom Pedro I,São Paulo,São Paulo,Brazil,"R. Dr. Mario Vicente, 1725 - Vila Dom Pedro I,...",street_address,-23.598837,-46.613584,ROOFTOP
